### Install MIP lib

In [ ]:
%%capture
!pip install mip
!pip install pulp
!pip install joblib

### Emulate local project structure

In [ ]:
pwd

In [ ]:
print ("Creating ./eit paper")

In [ ]:
mkdir ./eit_paper/

In [ ]:
cd ./eit_paper/

In [ ]:
ls

In [ ]:
!pwd

In [ ]:
print ("Copy Test instances here")
!cp ../../input/dual-exp/* .

In [ ]:
ls

In [ ]:
print ("Make src_dual/ src\ and copy respective files inside")

In [ ]:
mkdir src_dual src

In [ ]:
ls

In [ ]:
!cp ../../input/src-basic/* ./src/
!cp ../../input/src-dual/* ./src_dual/
!find $PWD | grep src_dual

In [ ]:
mkdir -p ./input/index-weekly-data/

In [ ]:
ls

In [ ]:
!cp ../../input/index-weekly-data/* ./input/index-weekly-data

In [ ]:
pwd

## Experiment starts

In [ ]:
base_path="../eit_basic_dual_unreduced_experiments/exp_2" #Don't add '/' at start and end
T=180 #Training time period
C=10000 #Capital available
file="9" #Index_file
#m=8 #Inital Size of Kernel
lbuck=10 #Size Of Buckets
#pho=0.1 #Transaction Cost Proportion
lamda=1/(100*C) # lower_bound for capital invested in jth stock
#nuh=0.3  # upper_bound
#xii=1.2 # Limit for Tracking Error
#k=20 #Cardinality constraint for stocks in portfolio
p=3 #If stock not selected in optimal soln in last p iters then it is to be dropped from kernel
f=12
w_return=100
w_risk=1000
w_risk_down=1.3
#cj,cb are declared in script

In [ ]:
import sys
from IPython.display import Image, display
sys.path.insert(0,"..")

In [ ]:
from eit_basic_experiment import TestEitBasic
from eit_dual_exp import TestEitDual

import time
from tqdm.notebook import tqdm
from IPython.utils import io
from joblib import Parallel, delayed
import pandas as pd

In [ ]:
!rm -r $base_path

In [ ]:
path_pattern=base_path+"/{}/file={},pho={},xii={},k={},m={},lbuck={},p={},nuh={}"

In [ ]:
%%time
basic_instances=[]
dual_instances=[]
for file in ["9"]:
    for pho in [0.25]:
        for xii in [1.1]:
            for k in [100]:
                for m in [70]:
                    for nuh in [0.2,0.25]:
                        output_basic=path_pattern.format("basic",file,pho,xii,k,m,lbuck,p,nuh)
                        #Create object for EIT-basic
                        test_eit_basic=TestEitBasic(output=output_basic,T=T,C=C,file=file,m=m,lbuck=lbuck,\
                                            pho=pho,lamda=lamda,nuh=nuh,xii=xii,k=k,p=p,f=f)
                        basic_instances.append(test_eit_basic)
                        #Create object for EIT-dual
                        output_dual=path_pattern.format("dual",file,pho,xii,k,m,lbuck,p,nuh)
                        test_eit_dual=TestEitDual(output=output_dual,T=T,C=C,file=file,m=m,lbuck=lbuck,pho=pho,\
                                                  lamda=lamda,nuh=nuh,xii=xii,k=k,p=p,f=f,w_risk=w_risk,\
                                                  w_return=w_return,w_risk_down=w_risk_down)
                        dual_instances.append(test_eit_dual)

In [ ]:
import traceback

In [ ]:
def parallel_func(eit_object):
    params=eit_object.give_params()
    step_dict={}
    try:
        result_df=eit_object.run_experiment(from_root=True,verbose=True);
        result_df["z_linear"]=eit_object.objective_linear
        #result_dict.update(step_dict)
    except Exception as e:
        print("Error for params",params)
        traceback.print_exc()
        result_df=None
    step_dict[params]=result_df   
    return (step_dict)

In [ ]:
#test_2={}
basic_outputs=Parallel(verbose = 2,prefer='processes', n_jobs=-1\
            )(delayed( parallel_func )(run) for run in tqdm(basic_instances))

In [ ]:
pwd

In [ ]:
%%time
#test_2={}
dual_outputs=Parallel(verbose = 2,prefer='processes', n_jobs=-1\
            )(delayed( parallel_func )(run) for run in tqdm(dual_instances))

In [ ]:
def make_result(parallel_output):
    final_result=pd.DataFrame()
    for output_i in parallel_output:
        params,result_df=next(iter(output_i.items()))
        param_part=pd.DataFrame([params])
        if result_df is None:
            res_part=pd.DataFrame()
        else:
            res_part=pd.DataFrame([result_df.loc[result_df.z_value.idxmax()]])
        temp=pd.concat([param_part,res_part.reset_index()],axis=1,ignore_index=False)
        final_result=final_result.append(temp)
    return (final_result)

In [ ]:
basic_result=make_result(basic_outputs)
basic_result.to_csv("./basic_results_exp_1.csv")

In [ ]:
basic_result

In [ ]:
dual_result=make_result(dual_outputs)
dual_result.to_csv("./dual_results_exp_1.csv")

In [ ]:
basic_result.columns

In [ ]:
print("Basic Infeasible for % of param combs={:.2f}",basic_result.z_value.isnull().mean())
print("Dual Infeasible for % of param combs={:.2f}",dual_result.z_value.isnull().mean())

In [ ]:
for var in ["in_tr_err","out_tr_err"]:
    print ("For basic -ve {}={:.3%}".format(var,(basic_result[var]<0).mean()))
    print ("For dual -ve {}={:.3%}".format(var,(dual_result[var]<0).mean()))

In [ ]:
basic_result["return_pu_risk_sample"]=basic_result["in_excess_return"]/basic_result["in_tr_err"].abs()
basic_result["return_pu_risk_oot"]=basic_result["out_excess_return"]/basic_result["out_tr_err"].abs()

dual_result["return_pu_risk_sample"]=dual_result["in_excess_return"]/dual_result["in_tr_err"].abs()
dual_result["return_pu_risk_oot"]=dual_result["out_excess_return"]/dual_result["out_tr_err"].abs()

In [ ]:
basic_k_pho_nuh=basic_result.groupby(["k","pho","nuh"])[["return_pu_risk_sample","return_pu_risk_oot"]].mean()
dual_k_pho_nuh=dual_result.groupby(["k","pho","nuh"])[["return_pu_risk_sample","return_pu_risk_oot"]].mean()

In [ ]:
from IPython.display import display_html 

In [ ]:
df1_styler = basic_k_pho_nuh.style.set_table_attributes("style='display:inline'").set_caption('EIT Basic')
df2_styler = dual_k_pho_nuh.style.set_table_attributes("style='display:inline'").set_caption('EIT Dual')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

In [ ]:
combined_k_pho_nuh=basic_k_pho_nuh.merge(dual_k_pho_nuh,on=["k","pho","nuh"],suffixes=("_basic","_dual"))

In [ ]:
combined_k_pho_nuh=basic_k_pho_nuh.merge(dual_k_pho_nuh,on=["k","pho","nuh"],suffixes=("_basic","_dual"))

In [ ]:
combined_k_pho_nuh.columns

In [ ]:
((combined_k_pho_nuh["return_pu_risk_sample_dual"]>combined_k_pho_nuh["return_pu_risk_sample_basic"])).mean()

In [ ]:
((combined_k_pho_nuh["return_pu_risk_oot_dual"]>combined_k_pho_nuh["return_pu_risk_oot_basic"])).mean()

In [ ]:
combined_k_pho_nuh.to_csv(base_path+"/combined_k_pho_nuh.csv")